<a href="https://colab.research.google.com/github/eropecrm/ChatGPT/blob/main/attacker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
https://colab.research.google.com/github/eropecrm/Adversarial-attack/blob/main/attacker.ipynb

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import copy
from abc import ABC, abstractmethod

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/AdversarialRobust-GAP-DSC

/content/drive/MyDrive/Colab Notebooks/AdversarialRobust-GAP-DSC


### 출처:
https://rain-bow.tistory.com/entry/%EC%A0%81%EB%8C%80%EC%A0%81-%EA%B3%B5%EA%B2%A9Adversarial-Attack-FGSMPGD

https://github.com/Jeffkang-94/pytorch-adversarial-attack/blob/master/attack/Attacker.py


In [ ]:
def fgsm_attack(image, epsilon, gradient): # gradient 의 요소별 부호 값을 얻어옵니다
    # 기울기값의 원소의 sign 값을 구함, 입력 이미지의 각 픽셀에 sign_data_gradient를 적용해 작은 변화가 적용된 이미지를 생성.
    sign_gradient = gradient.sign()
    # 이미지 각 픽셀의 값을 sign_gradient 방향으로 epsilon 만큼 조절
    perturbed_image = image + epsilon * sign_gradient
    # [0,1] 범위를 벗어나는 값을 조절, 값 범위를 [0,1]로 유지하기 위해 자르기(clipping)를 추가
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    # 작은 변화가 적용된 이미지를 리턴
    return perturbed_image

In [ ]:
def pgd_attack(model, model_number, images, labels,loss,  device, batch_size, eps=0.1, alpha=2 / 255, attack_steps = 7):
    images = images.to(device)
    labels = labels.to(device)

    ori_images = images.data

    for i in range(iters):

        images.requires_grad = True

        outputs = model.forward(images, batch_size).requires_grad_()
        '''
        for i in range (0,10):
            model.set_zerograd(i)
            '''
        model.zero_grad()
        cost = loss(outputs, labels).to(device)
        cost.backward()

        #gradient =model.get_sep_gredient(model_number=model_number,x=images,y_=labels,batch_size=batch_size)
        adv_images = images + alpha * images.grad.data
        eta = torch.clamp(adv_images - ori_images, min=-eps, max=eps)
        images = torch.clamp(ori_images + eta, min=0, max=1).detach_()

    return images

In [ ]:
# CW-L2 Attack
# https://github.com/Harry24k/CW-pytorch/blob/master/CW.ipynb
# Based on the paper, i.e. not exact same version of the code on https://github.com/carlini/nn_robust_attacks
# (1) Binary search method for c, (2) Optimization on tanh space, (3) Choosing method best l2 adversaries is NOT IN THIS CODE.
def cw_l2_attack(model, images, labels, targeted=False, c=1e-4, kappa=0, max_iter=1000, learning_rate=0.01) :

    images = images.to(device)     
    labels = labels.to(device)

    # Define f-function
    def f(x) :

        outputs = model(x)
        one_hot_labels = torch.eye(len(outputs[0]))[labels].to(device)

        i, _ = torch.max((1-one_hot_labels)*outputs, dim=1)
        j = torch.masked_select(outputs, one_hot_labels.byte())
        
        # If targeted, optimize for making the other class most likely 
        if targeted :
            return torch.clamp(i-j, min=-kappa)
        
        # If untargeted, optimize for making the other class most likely 
        else :
            return torch.clamp(j-i, min=-kappa)
    
    w = torch.zeros_like(images, requires_grad=True).to(device)

    optimizer = optim.Adam([w], lr=learning_rate)

    prev = 1e10
    
    for step in range(max_iter) :

        a = 1/2*(nn.Tanh()(w) + 1)

        loss1 = nn.MSELoss(reduction='sum')(a, images)
        loss2 = torch.sum(c*f(a))

        cost = loss1 + loss2

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        # Early Stop when loss does not converge.
        if step % (max_iter//10) == 0 :
            if cost > prev :
                print('Attack Stopped due to CONVERGENCE....')
                return a
            prev = cost
        
        print('- Learning Progress : %2.2f %%        ' %((step+1)/max_iter*100), end='\r')

    attack_images = 1/2*(nn.Tanh()(w) + 1)

    return attack_images

### 4. Class 정의

In [ ]:
class Attacker:
    def __init__(self, clip_max=0.5, clip_min=-0.5):
        self.clip_max = clip_max
        self.clip_min = clip_min

    def generate(self, model, x, y):
        pass

In [ ]:
class BIM(Attacker):

    def __init__(self, eps=0.15, eps_iter=0.01, n_iter=50, clip_max=0.5, clip_min=-0.5):
        super(BIM, self).__init__(clip_max, clip_min)
        self.eps = eps
        self.eps_iter = eps_iter
        self.n_iter = n_iter

    def generate(self,model, device, lossf, batchsize,  x, y):
        model.eval()
        nx = x
        ny = y
        #nx = torch.unsqueeze(x, 0)
        #ny = torch.unsqueeze(y, 0)
        nx.requires_grad_()
        eta = torch.zeros(nx.shape).to(device)

        for i in range(self.n_iter):
            out = model.forward(nx + eta, batchsize)
            loss = lossf(out, ny)
            #loss = F.cross_entropy(out, ny)
            loss.backward()

            eta += self.eps_iter * torch.sign(nx.grad.data)
            eta.clamp_(-self.eps, self.eps)
            nx.grad.data.zero_()

        x_adv = nx + eta
        x_adv.clamp_(self.clip_min, self.clip_max)
        x_adv.squeeze_(0)

        return x_adv.detach()


In [ ]:
class GAP(nn.Module):
    def __init__(self):
        super(GAP, self).__init__()

    def forward(self, x):
        return torch.mean(x.view(x.size(0), x.size(1), -1), dim=2)